# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

#####Importing Tensprflow and Keras libraries:

In [0]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


#####Importing Cifar10 library:

In [0]:
(x_train,y_train),(x_test,y_test)=keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
index_0to4 = np.where(y_train<5)[0]
x_train_0to4 = x_train[index_0to4]
y_train_0to4 = y_train[index_0to4]
print(x_train_0to4.shape,y_train_0to4.shape)

(25000, 32, 32, 3) (25000, 1)


In [0]:
index_5to9 = np.where(y_train>4)[0]
x_train_5to9 = x_train[index_5to9]
y_train_5to9 = y_train[index_5to9]
print(x_train_5to9.shape,y_train_5to9.shape)

(25000, 32, 32, 3) (25000, 1)


In [0]:
index_test_0to4 = np.where(y_test<5)[0]
x_test_0to4 = x_test[index_test_0to4]
y_test_0to4 = y_test[index_test_0to4]
print(x_test_0to4.shape,y_test_0to4.shape)

(5000, 32, 32, 3) (5000, 1)


In [0]:
index_test_5to9 = np.where(y_test>4)[0]
x_test_5to9 = x_test[index_test_5to9]
y_test_5to9 = y_test[index_test_5to9]
print(x_test_5to9.shape,y_test_5to9.shape)

(5000, 32, 32, 3) (5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

#####Dividing given data into Train and Test data to apply One-hot encoding:

In [0]:
y_train_0to4_enc = keras.utils.to_categorical(y_train_0to4)
y_train_5to9_enc = keras.utils.to_categorical(y_train_5to9)[:,5:]
y_test_0to4_enc = keras.utils.to_categorical(y_test_0to4)
y_test_5to9_enc = keras.utils.to_categorical(y_test_5to9)[:,5:]

In [0]:
print(y_train_0to4_enc.shape,y_train_5to9_enc.shape)
print(y_test_0to4_enc.shape,y_test_5to9_enc.shape)

(25000, 5) (25000, 5)
(5000, 5) (5000, 5)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout
from keras import callbacks

#####Normalizing the inputs from 0-255 to 0.0-1.0

##### Initializing Sequential Model:

In [0]:
model_0to4 = Sequential()

##### Additing 1st convolution layer:

In [0]:
model_0to4.add(Conv2D(24,(3,3),input_shape=(32,32,3),activation='relu'))

##### Additing MaxPooling layer:

In [0]:
model_0to4.add(MaxPool2D(pool_size=(2,2)))

##### Additing 2nd convolution layer:

In [0]:
model_0to4.add(Conv2D(256,(3,3),activation='relu'))

##### Addition of 3rd Covolution layer:

In [0]:
model_0to4.add(Conv2D(512,(3,3),activation = 'relu'))

##### Addition of MaxPooling layer

In [0]:
model_0to4.add(MaxPool2D(pool_size=(2,2)))

##### Flatten the results for dense layer

In [0]:
model_0to4.add(Flatten())

##### Adding 1st Dense layer:

In [0]:
model_0to4.add(Dense(500,activation='sigmoid'))

##### Adding 15% dropout layer:

In [0]:
model_0to4.add(Dropout(0.35))

##### Adding 3rd Dense layer:

In [0]:
model_0to4.add(Dense(150,activation='sigmoid'))

##### Adding 4th Dense layer:

In [0]:
model_0to4.add(Dense(50,activation='sigmoid'))

#####Adding output layer:

In [0]:
model_0to4.add(Dense(5,activation='softmax'))

In [0]:
sgd_opt = keras.optimizers.SGD(lr=0.02)
model_0to4.compile(loss='categorical_crossentropy',optimizer=sgd_opt,metrics=['accuracy'])

##### Monitoring val_acc and stop training when it stops improving:

In [0]:
callback_list = [callbacks.EarlyStopping(monitor='val_acc',patience=5,verbose=1,mode='auto')]

model_0to4.fit(x_train_0to4,y_train_0to4_enc,batch_size=32,epochs=50,callbacks=callback_list,
               validation_data=(x_test_0to4,y_test_0to4_enc))

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 14s 579us/step - loss: 1.4662 - acc: 0.3778 - val_loss: 1.4126 - val_acc: 0.3890
Epoch 2/50
25000/25000 [==============================] - 11s 438us/step - loss: 1.1503 - acc: 0.5436 - val_loss: 0.9941 - val_acc: 0.5996
Epoch 3/50
25000/25000 [==============================] - 11s 437us/step - loss: 0.9563 - acc: 0.6264 - val_loss: 0.9961 - val_acc: 0.6142
Epoch 4/50
25000/25000 [==============================] - 11s 435us/step - loss: 0.8368 - acc: 0.6727 - val_loss: 1.0025 - val_acc: 0.5912
Epoch 5/50
25000/25000 [==============================] - 11s 434us/step - loss: 0.7406 - acc: 0.7188 - val_loss: 1.0965 - val_acc: 0.5546
Epoch 6/50
25000/25000 [==============================] - 11s 430us/step - loss: 0.6507 - acc: 0.7591 - val_loss: 0.7362 - val_acc: 0.7112
Epoch 7/50
25000/25000 [==============================] - 11s 429us/step - loss: 0.5783 - acc: 0.7844 - val_loss: 0.6

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
model_0to4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_118 (Conv2D)          (None, 30, 30, 24)        672       
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 15, 15, 24)        0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 13, 13, 256)       55552     
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 11, 11, 512)       1180160   
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 5, 5, 512)         0         
_________________________________________________________________
flatten_42 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_168 (Dense)            (None, 500)               6400500   
__________

In [0]:
for layer in model_0to4.layers:
  if(type(layer) == Conv2D) :
    layer.trainable=False

In [0]:
model_0to4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_118 (Conv2D)          (None, 30, 30, 24)        672       
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 15, 15, 24)        0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 13, 13, 256)       55552     
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 11, 11, 512)       1180160   
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 5, 5, 512)         0         
_________________________________________________________________
flatten_42 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_168 (Dense)            (None, 500)               6400500   
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

##### Monitoring val_acc and stop training when it stops improving:

In [0]:
callback_list = [callbacks.EarlyStopping(monitor='val_acc',patience=5,verbose=1,mode='auto')]

model_0to4.fit(x_train_5to9,y_train_5to9_enc,batch_size=32,epochs=50,callbacks=callback_list,
               validation_data=(x_test_5to9,y_test_5to9_enc))

Train on 25000 samples, validate on 5000 samples
Epoch 1/50
  320/25000 [..............................] - ETA: 14s - loss: 4.7552 - acc: 0.2469

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 11s 442us/step - loss: 0.8763 - acc: 0.6837 - val_loss: 0.6756 - val_acc: 0.7480
Epoch 2/50
25000/25000 [==============================] - 11s 434us/step - loss: 0.5387 - acc: 0.7997 - val_loss: 0.5640 - val_acc: 0.7896
Epoch 3/50
25000/25000 [==============================] - 11s 437us/step - loss: 0.4237 - acc: 0.8475 - val_loss: 0.4839 - val_acc: 0.8238
Epoch 4/50
25000/25000 [==============================] - 11s 437us/step - loss: 0.3324 - acc: 0.8806 - val_loss: 1.3546 - val_acc: 0.6158
Epoch 5/50
25000/25000 [==============================] - 11s 431us/step - loss: 0.2681 - acc: 0.9036 - val_loss: 0.6287 - val_acc: 0.7900
Epoch 6/50
25000/25000 [==============================] - 11s 432us/step - loss: 0.1926 - acc: 0.9332 - val_loss: 0.4909 - val_acc: 0.8332
Epoch 7/50
25000/25000 [==============================] - 11s 430us/step - loss: 0.1364 - acc: 0.9533 - val_loss: 0.5284 - val_acc: 0.8290
Epoch 8/50
25000/25000 [==============

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

I0622 16:34:20.375514 140637328041856 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0622 16:34:20.378869 140637328041856 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

##### Using the TfIDFVectorizer to create document-term matrices:

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer()
train_X = vectorizer.fit_transform(twenty_train.data).toarray()
train_Y = twenty_train.target
test_X = vectorizer.transform(twenty_test.data).toarray()
test_Y = twenty_test.target

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

##### importing and instantiating a logistic regression model:

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
vector_model = LogisticRegression()
vector_model.fit(train_X,train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
print("Train accuracy score:- ",vector_model.score(train_X,train_Y))
print("Test accuracy score:- ",vector_model.score(test_X,test_Y))

Train accuracy score:-  0.9827204253433761
Test accuracy score:-  0.8868175765645806
